In [7]:
import pandas as pd
df = pd.read_csv(r'D:\DataScience Revision\Project(Capstone Project)\Customer Churn Prediction\data\raw\telecom_churn.csv')
df['Churn'].value_counts(normalize=True)

Churn
0    0.95254
1    0.04746
Name: proportion, dtype: float64

In [8]:
df.head()
df.shape
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Monthly_Bill            100000 non-null  float64
 1   Total_Data_Usage        100000 non-null  float64
 2   Customer_Service_Calls  100000 non-null  int64  
 3   Contract_Type           100000 non-null  int64  
 4   Churn                   100000 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 3.8 MB


In [9]:
df['Contract_Type'].value_counts()


Contract_Type
0    59893
1    40107
Name: count, dtype: int64

In [11]:
df.describe()


,Monthly_Bill,Total_Data_Usage,Customer_Service_Calls,Contract_Type,Churn
count,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,89.93858,253.158414,1.503900,0.401070,0.047460
std,34.60098,142.902402,1.224173,0.490118,0.212622
min,30.00000,5.000000,0.000000,0.000000,0.000000
25%,59.95000,129.107500,1.000000,0.000000,0.000000
50%,90.07500,253.430000,1.000000,0.000000,0.000000
75%,119.94000,376.455000,2.000000,1.000000,0.000000
max,150.00000,499.990000,9.000000,1.000000,1.000000


In [12]:
df.groupby('Churn').mean()


,Monthly_Bill,Total_Data_Usage,Customer_Service_Calls,Contract_Type
Churn,,,,
0,89.905851,257.544447,1.495370,0.412455
1,90.595464,165.129094,1.675095,0.172566


In [13]:
df.value_counts('Contract_Type', normalize=True)

Contract_Type
0    0.59893
1    0.40107
Name: proportion, dtype: float64

In [14]:
X = df.drop('Churn', axis=1)
y = df['Churn']


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)

model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [18]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.98      0.67      0.79     19051
           1       0.09      0.67      0.16       949

    accuracy                           0.67     20000
   macro avg       0.53      0.67      0.48     20000
weighted avg       0.93      0.67      0.76     20000

ROC-AUC: 0.7273938143629665


In [19]:
import numpy as np
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

# مثال Threshold مناسب
threshold = 0.3
y_pred_custom = (y_prob >= threshold).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

           0       0.99      0.32      0.48     19051
           1       0.06      0.93      0.12       949

    accuracy                           0.35     20000
   macro avg       0.53      0.62      0.30     20000
weighted avg       0.94      0.35      0.47     20000



In [20]:
for t in [0.3, 0.35, 0.4]:
    y_pred_t = (y_prob >= t).astype(int)
    print(f"\nThreshold = {t}")
    print(classification_report(y_test, y_pred_t))



Threshold = 0.3
              precision    recall  f1-score   support

           0       0.99      0.32      0.48     19051
           1       0.06      0.93      0.12       949

    accuracy                           0.35     20000
   macro avg       0.53      0.62      0.30     20000
weighted avg       0.94      0.35      0.47     20000


Threshold = 0.35
              precision    recall  f1-score   support

           0       0.99      0.42      0.59     19051
           1       0.07      0.88      0.13       949

    accuracy                           0.44     20000
   macro avg       0.53      0.65      0.36     20000
weighted avg       0.94      0.44      0.56     20000


Threshold = 0.4
              precision    recall  f1-score   support

           0       0.98      0.50      0.66     19051
           1       0.07      0.81      0.14       949

    accuracy                           0.51     20000
   macro avg       0.53      0.66      0.40     20000
weighted avg       0.9